In [1]:
import tensorflow as tf

for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Other imports for processing data
import string
import numpy as np
import pandas as pd

In [14]:
dataset = pd.read_csv('../datasets/songdata.csv', dtype=str)[:250]
field = 'text'
dataset[field] = dataset[field].str.replace('[{}]'.format(string.punctuation), '')
dataset[field] = dataset[field].str.lower()
sentences = dataset['text'].str.cat().split('\n')
sentences = [x.rstrip() for x in sentences]
sentences = [x for x in sentences if x!= '']

C:\Users\sebas\AppData\Local\Temp\ipykernel_6764\2035965103.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset[field] = dataset[field].str.replace('[{}]'.format(string.punctuation), '')


In [31]:
tokenizer = Tokenizer(num_words=2000,oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

sequences = []
for line in sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)

maxlen = max([len(x) for x in sequences])

sequences = np.array(pad_sequences(sequences,padding='pre',maxlen=maxlen))

input_sequences, labels = sequences[:,:-1], sequences[:,-1]

one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes=tokenizer.num_words)

In [32]:
input_sequences

array([[  0,   0,   0, ...,   0,   0, 112],
       [  0,   0,   0, ...,   0, 112,  67],
       [  0,   0,   0, ..., 112,  67,  87],
       ...,
       [  0,   0,   0, ...,  16, 568, 138],
       [  0,   0,   0, ..., 568, 138,  94],
       [  0,   0,   0, ...,   0,   0, 953]])